In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [25]:
train_dir = '../data/train/'
test_dir = '../data/test/'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)



Found 1476 images belonging to 2 classes.
Found 1776 images belonging to 2 classes.


In [26]:
base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)



In [27]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [28]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)


Epoch 1/3
47/47 ━━━━━━━━━━━━━━━━━━━━ 533s 10s/step - accuracy: 0.8939 - loss: 0.2896 - val_accuracy: 0.5000 - val_loss: 6138.3716
Epoch 2/3


C:\Users\masub\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/3
47/47 ━━━━━━━━━━━━━━━━━━━━ 506s 11s/step - accuracy: 0.9749 - loss: 0.1335 - val_accuracy: 0.5000 - val_loss: 7.8295


In [33]:
import numpy as np
from tensorflow.keras.preprocessing import image
import cv2

# Load and preprocess an image
img_path = 'data/myimage/image_1.png'

# Load the image using OpenCV
img = cv2.imread(img_path)

# Resize the image to the required input size of the model (224x224) and convert to grayscale
img = cv2.resize(img, (224, 224))
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Convert grayscale image to float32
gray_img = gray_img.astype('float32')

# Normalize the image
normalized_img = gray_img / 255.0

# Convert grayscale image to three-channel (RGB) image
rgb_img = cv2.cvtColor(normalized_img, cv2.COLOR_GRAY2RGB)

# Expand dimensions to match the input shape expected by the model
x = np.expand_dims(rgb_img, axis=0)

# Predict
prediction = model.predict(x)

# Visualize result
if prediction < 0.5:
    print("The image is predicted to be normal.")
else:
    print("The image is predicted to have cataract.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
The image is predicted to be normal.


In [34]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess an image
img_path = 'data/myimage/image_1.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.

# Predict
prediction = model.predict(x)

# Visualize result
if prediction < 0.5:
    print("The image is predicted to be normal.")
else:
    print("The image is predicted to have cataract.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
The image is predicted to be normal.
